In [1]:
import os, sys
import pandas as pd
from dotenv import load_dotenv

sys.path.append(os.path.abspath(".."))

load_dotenv("../.env", override=True)



True

In [2]:
import importlib
import src.api as api
importlib.reload(api)

<module 'src.api' from '/Users/marisa/Desktop/Data Analytics/Week7/Tech-Salary-Benchmarking-Cross-Market-Analysis-for-Competitive-Salary-Bands/src/api.py'>

In [10]:
import time
from pathlib import Path
import pandas as pd

PRIMARY_TITLES = [
    "Software Developer",
    "Software Engineer",
    "Full Stack Web Developer",
    "Mobile Developer",
    "UX Designer",
    "QA Engineer",
    "Backend Engineer",
    "Frontend Engineer",
]

LOCATIONS = [
    "London, UK",
    "Lisbon, Portugal",
    "Malmo, Sweden",
    "New York, USA",
]

LEVEL_TO_YEARS = {
    "Junior": 0,
    "Mid": 3,
    "Senior": 5,
    "Lead": 7,
    "Principal": 9,
    "Head of": 12,
}

In [12]:
calls_made = 0
all_rows = []

first_big_run = True  

for title in PRIMARY_TITLES:
    for loc in LOCATIONS:
        for level_name, years in LEVEL_TO_YEARS.items():

            try:
                payload = api.fetch_role_location(
                    role=title,
                    location=loc,
                    years_experience=years,
                    force_refresh=first_big_run
                )
            except Exception as e:
                print("FAILED:", title, loc, level_name, "|", repr(e))
                time.sleep(2)
                continue

            rows = api.extract_rows(payload)
            if not rows:
                calls_made += 1
                time.sleep(1)
                continue

            for r in rows:
                r["level_name"] = level_name
                r["years_experience"] = years
                r["api_title_requested"] = title
                r["api_location_requested"] = loc

            all_rows.extend(rows)
            calls_made += 1

            time.sleep(1)  

    first_big_run = False

print("Calls made this run:", calls_made)
df_api_raw = pd.DataFrame(all_rows)
df_api_raw.head()

Calls made this run: 192


,job_title,location,min_salary,max_salary,avg_salary,currency,salary_period,confidence,publisher_name,level_name,years_experience,api_title_requested,api_location_requested
0,Software Developer,"Lisbon, Portugal",1612.5,2679.166667,None,None,MONTH,VERY_HIGH,Glassdoor,Junior,0,Software Developer,"Lisbon, Portugal"
1,Software Developer,"Lisbon, Portugal",1612.5,2679.166667,None,None,MONTH,VERY_HIGH,Glassdoor,Mid,3,Software Developer,"Lisbon, Portugal"
2,Software Developer,"Lisbon, Portugal",1612.5,2679.166667,None,None,MONTH,VERY_HIGH,Glassdoor,Senior,5,Software Developer,"Lisbon, Portugal"
3,Software Developer,"Lisbon, Portugal",1612.5,2679.166667,None,None,MONTH,VERY_HIGH,Glassdoor,Lead,7,Software Developer,"Lisbon, Portugal"
4,Software Developer,"Lisbon, Portugal",1612.5,2679.166667,None,None,MONTH,VERY_HIGH,Glassdoor,Principal,9,Software Developer,"Lisbon, Portugal"


In [14]:
from pathlib import Path

PROCESSED_DIR = Path("data/processed")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

df_api_raw.to_csv(PROCESSED_DIR / "api_raw_6levels.csv", index=False)
print("Saved:", PROCESSED_DIR / "api_raw_6levels.csv", df_api_raw.shape)


Saved: data/processed/api_raw_6levels.csv (138, 13)


In [13]:
from pathlib import Path

PROCESSED_DIR = Path("data/processed")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

df_api_raw.to_csv(PROCESSED_DIR / "api_raw_bigpull.csv", index=False)
print("Saved:", PROCESSED_DIR / "api_raw_bigpull.csv", df_api_raw.shape)

Saved: data/processed/api_raw_bigpull.csv (138, 13)
